In [21]:
import time
t1 = time.time()
user_play_1 = pd.read_excel("附件1：用户收视信息.xlsx", encoding= 'gb18030', sheetname="用户点播信息")
user_play_2 = pd.read_excel("附件1：用户收视信息.xlsx", encoding= 'gb18030', sheetname="用户单片点播信息") 
user_play_2.rename(columns={"影片名称":"节目名称"}, inplace=True)
all_program = pd.read_csv("item_info.csv", encoding="GB18030")

In [22]:
import re
import pandas as pd
pd.set_option('display.max_columns', None)

def clean_by_pattern(df):
    pattern = "^\d+\-\d+(\w+)?"
    df[pattern] = [re.findall(pattern, i) for i in df["节目名称"]]
    df[pattern+"_true"] = [len(i)!=0 for i in df[pattern]]
    df.loc[df[pattern+"_true"], "节目名称"] = [i[0] for i in df.loc[df[pattern+"_true"], pattern]]
    df.drop([pattern, "^\d+\-\d+(\w+)?_true"], axis=1, inplace=True)
    return df

def clean_program_name(df):
    '''
    清洗节目名称的函数
    '''
    
    key_words = ["甜果学堂", "花城全视角", "花开花城", "优·秀", "体育竞技广告", "甜果屋","甜果课堂", "今天我上线", "兔智来了", "神奇萝卜", "舌尖上的中国Ⅱ", 
             "开心超人联盟", "舌尖上的中国", "星际小蚂蚁", "小小羊提米", "食得健康吗", "阿U", "经典广告", "爆裂飞车", "赌城群英会", 
             "综艺娱乐广告", "维多利亚的秘密", "开心球", "食得健康吗", "大鹏嘚吧嘚", "黑子的篮球", "斗龙战士4", "西游记", "西游记续", 
            "生肖传奇", "那年花开月正圆", "家庭影院广告", "频道一广告", "宣传频道广告", "野外独游", "改变世界的武器", "厝边头尾", 
            "圣龙的守护", "启源星之战", "战神崛起", "最终幻想", "头号公敌", "赛尔号4", "贪狼",
            "关晓彤生日会", "CP","170415平安365", "电视剧场广告", "我的电视广告"]
    
    
    # 去掉名称前后的括号() 
    df["节目名称"] = [re.sub("\(\S*?\)+?", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("\(.*", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("（\S*?）+?", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("（\S*?\)+?", "", i) for i in df["节目名称"]]
    # 去掉名称前的日期 
    for i in df["节目名称"].index: 
        if re.findall(r"月(.+)日 ?(.+)", df.loc[i, "节目名称"]): 
                df.loc[i, "节目名称"] = re.search(r"月(.+)日 ?(.+)", df.loc[i, "节目名称"]).group(2) 
        if "】" in df.loc[i, "节目名称"]: 
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split("】")[1]
        if "：" in df.loc[i, "节目名称"]:
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split("：")[0]
        if " "  in df.loc[i, "节目名称"]:  
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split(" ")[0]
        
        if "《" in df.loc[i, "节目名称"]: 
                df.loc[i, "节目名称"] = re.search(r"《(.+)》?(.+)", df.loc[i, "节目名称"]).group(1) 
                
        for key_word in key_words:
            if key_word in df.loc[i, "节目名称"]:
                df.loc[i, "节目名称"] = key_word    
    df = clean_by_pattern(df)
    df.drop(df.loc[df["节目名称"].str.contains("测试")].index, inplace=True)
    

def to_diclist(df):
    '''
    将节目信息转成字典列表
    '''
    new_tfv = df
    new_tfv_list = []
    for i in new_tfv.index:
        new_tfvs = {}
        new_tfvs["节目名称"] = new_tfv["节目名称"][i]
        new_tfvs["类型"] = new_tfv["类型"][i]
        new_tfvs["形式"] = new_tfv["形式"][i]
        new_tfvs["声道语种"] = new_tfv["声道语种"][i]
        new_tfvs["地区参数"] = new_tfv["地区参数"][i]
        new_tfv_list.append(new_tfvs)
    return new_tfv_list



def add_feature(all_user):
    for i in list(set(all_user["节目名称"])):
        if i in list(set(all_program["节目名称"])):
            all_user.loc[all_user["节目名称"] == i, "形式"] = all_program.loc[all_program["节目名称"]== i ]["形式"].values[0]
            all_user.loc[all_user["节目名称"] == i, "地区参数"] = all_program.loc[all_program["节目名称"]== i ]["地区参数"].values[0]
            all_user.loc[all_user["节目名称"] == i, "声道语种"] = all_program.loc[all_program["节目名称"]== i ]["声道语种"].values[0]
            all_user.loc[all_user["节目名称"] == i, "类型"] = all_program.loc[all_program["节目名称"]== i ]["类型"].values[0]
    return all_user

In [23]:
clean_program_name(user_play_1)
clean_program_name(user_play_2)
user_play_1.drop_duplicates(inplace=True)
user_play_2.drop_duplicates(inplace=True)
user_1 = user_play_1[["设备号", "节目名称"]]
user_2 = user_play_2[["设备号", "节目名称"]]
user_program = user_1.append(user_2)
user_program.reset_index(drop=True, inplace=True)

t2 = time.time()
print("用时：", t2-t1)

用时： 132.5978775024414


# 判断用户点播的节目中是否有新产品

####【循环】

In [5]:
#判断是否有新的电视产品
new_prod = []
user_tfv = list(set(user_program["节目名称"]))
for i in user_tfv:
    if i not in list(all_program["节目名称"]):
        new_prod.append(i)

In [6]:
len(new_prod) #空列表则表示没有新节目，就可以跳过【添加新节目到数据库】这一步

0

### 【添加新节目到数据库】

In [85]:
#若有新节目，则执行下面两步操作
import requests
from lxml import etree
from fake_useragent import UserAgent


# 代理服务器
proxyHost = "http-cla.abuyun.com"
proxyPort = "9030"

# 代理隧道验证信息
proxyUser = "HKB8867XIP4PI1EC"
proxyPass = "3FA0788983ACC3C6"

proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
      "host" : proxyHost,
      "port" : proxyPort,
      "user" : proxyUser,
      "pass" : proxyPass,
    }

proxies = {
        "http"  : proxyMeta,
        "https" : proxyMeta,
    }



headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36"
           
          }

def parse_url(url):
    try:
        r = requests.get(url, headers=headers,proxies=proxies, timeout =20)
        r.raise_for_status()
        response = r.content.decode()
        html = etree.HTML(response)
        return html
    except:
        print("error")

def get_tfv_info(name):
    name = name
    url = "https://www.douban.com/search?cat=1002&q={}".format(name)
    name_html = parse_url(url)
    name_url = name_html.xpath("//*[@id='content']/div/div[1]/div[3]/div[2]/div[1]/div[2]/div/h3/a/@href")
    form = name_html.xpath("//*[@id='content']/div/div[1]/div[3]/div[2]/div[1]/div[2]/div/h3/span[1]/text()")
    
    tfv ={}
    tfv["节目名称"] = name
    #形式
    if form:
        tfv["形式"] = form[0][1:-1]
    else:
        tfv["形式"] = ""
        
    
    try:
        info_html = parse_url(name_url[0])
        lst_temp = info_html.xpath("//*[@id='info']/text()")
        str_temp = "".join(lst_temp).strip()
        tags = info_html.xpath("//div[@id='info']/span[@property='v:genre']/text()")
        tfv["类型"] = tags
        tfv["地区参数"] = re.search("[日本香港台湾中韩美法英]+国?", str_temp).group(0)
        tfv["声道语种"] = re.search("[汉日英粤韩法]语+", str_temp).group(0)
    except:
        tfv["类型"] = ""
        tfv["地区参数"] = "其他地区"
        tfv["声道语种"] = "其他"
    
    new_tfv_list.append(tfv)
    print(tfv)

In [88]:
import time
from multiprocessing.dummy import Pool as ThreadPool

new_tfv_list = []
start = time.time()
pool = ThreadPool(3)  #设置并发数量
pool.map(get_tfv_info,list(new_prod))
pool.close() 
pool.join()
end = time.time()
print("用时：",end-start)

{'节目名称': '野外独游', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '丧子疑云', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '蛇胆雄威', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '爵士兔之奇幻之旅', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '设计来你家', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '烹饪的艺术', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '空袭生死线', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '毁灭前夕', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '挑战王者', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '小小羊提米提米的宠物问题', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '人在?逋？去拖影)', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '五彩缤纷的旅途', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '芭芭拉', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '?潘磕惺？第二季', '形式': '', '类型': '', '地区参数': '其他地区', '声道语种': '其他'}
{'节目名称': '新年行动', '形式': '', '类型': '',

In [91]:
new_tfv = pd.DataFrame(new_tfv_list)

#首先处理语种，将汉语改为国语，其他的看情况修改
new_tfv.loc[new_tfv["声道语种"]=="汉语", "声道语种"] = "国语"

#再处理地区参数
new_tfv.loc[new_tfv["地区参数"]=="英", "地区参数"] = "英国"
new_tfv.loc[new_tfv["地区参数"]=="中国", "地区参数"] = "大陆"

In [78]:
new_tfv.loc[new_tfv["节目名称"].str.contains("小蚂蚁"), "类型"] = [['少儿']]
new_tfv.loc[new_tfv["节目名称"].str.contains("小蚂蚁"), "形式"] = "科学教育"
new_tfv.loc[new_tfv["节目名称"].str.contains("小蚂蚁"), "地区参数"] = "大陆"
new_tfv.loc[new_tfv["节目名称"].str.contains("小蚂蚁"), "声道语种"] = "国语"

new_tfv.loc[new_tfv["节目名称"].str.contains("演唱会"), "类型"] = [['Live']]
new_tfv.loc[new_tfv["节目名称"].str.contains("演唱会"), "形式"] = "音乐"
new_tfv.loc[new_tfv["节目名称"].str.contains("演唱会"), "地区参数"] = "大陆"
new_tfv.loc[new_tfv["节目名称"].str.contains("演唱会"), "声道语种"] = "国语"

new_tfv.loc[new_tfv["节目名称"].str.contains("甜果"), "类型"] = [['生活']]
new_tfv.loc[new_tfv["节目名称"].str.contains("甜果"), "形式"] = "生活服务"
new_tfv.loc[new_tfv["节目名称"].str.contains("甜果"), "地区参数"] = "大陆"
new_tfv.loc[new_tfv["节目名称"].str.contains("甜果"), "声道语种"] = "国语"

new_tfv.loc[new_tfv["节目名称"].str.contains("年级"), "类型"] = [['教育']]
new_tfv.loc[new_tfv["节目名称"].str.contains("年级"), "形式"] = "生活服务"
new_tfv.loc[new_tfv["节目名称"].str.contains("年级"), "地区参数"] = "大陆"
new_tfv.loc[new_tfv["节目名称"].str.contains("年级"), "声道语种"] = "国语"



In [87]:
new_tfv = new_tfv.loc[new_tfv['类型']!=""]
new_tfv.reset_index(drop=True, inplace=True)

In [80]:
#将完善信息后的新节目插入数据库中
#new_program = to_diclist(new_tfv)
#program_collection.insert_many(new_program)

all_program = pd.read_csv("all_program.csv", encoding="GB18030")
new_all_program = all_program.append(new_tfv)
new_all_program.reset_index(drop=True, inplace=True)

new_all_program.to_csv("all_program.csv", index=False)

####【循环】

### 无法从豆瓣爬取的节目手动添加

In [144]:
#根据形式来看有哪些节目没有形式和类型，然后手动添加
new_tfv["形式"].value_counts()
#new_tfv.loc[new_tfv["形式"]==""]
new_tfv.loc[new_tfv["类型"]==""]

,地区参数,声道语种,形式,类型,节目名称


In [136]:
new_tfv.loc[new_tfv["节目名称"].str.contains("家家行运菜"), "类型"] = [['生活']]
new_tfv.loc[new_tfv["节目名称"].str.contains("家家行运菜"), "形式"] = "生活服务"
new_tfv.loc[new_tfv["节目名称"].str.contains("家家行运菜"), "地区参数"] = "大陆"
new_tfv.loc[new_tfv["节目名称"].str.contains("家家行运菜"), "声道语种"] = "国语"

In [142]:
new_tfv.loc[new_tfv["节目名称"].str.contains("桑德森实验"), "类型"] = [['科幻','惊悚']]
new_tfv.loc[new_tfv["节目名称"].str.contains("桑德森实验"), "形式"] = "电影"
new_tfv.loc[new_tfv["节目名称"].str.contains("桑德森实验"), "地区参数"] = "美国"
new_tfv.loc[new_tfv["节目名称"].str.contains("桑德森实验"), "声道语种"] = "英语"

In [141]:
new_tfv.loc[new_tfv["节目名称"].str.contains("谜一样的双眼"), "类型"] = [['剧情','爱情','悬疑']]
new_tfv.loc[new_tfv["节目名称"].str.contains("谜一样的双眼"), "形式"] = "电影"
new_tfv.loc[new_tfv["节目名称"].str.contains("谜一样的双眼"), "地区参数"] = "其他地区"
new_tfv.loc[new_tfv["节目名称"].str.contains("谜一样的双眼"), "声道语种"] = "其他"

In [138]:
new_tfv.loc[new_tfv["节目名称"].str.contains("宝莱坞机器人之恋"), "类型"] = [['喜剧','动作', '科幻','冒险','奇幻']]
new_tfv.loc[new_tfv["节目名称"].str.contains("宝莱坞机器人之恋"), "形式"] = "电影"
new_tfv.loc[new_tfv["节目名称"].str.contains("宝莱坞机器人之恋"), "地区参数"] = "其他地区"
new_tfv.loc[new_tfv["节目名称"].str.contains("宝莱坞机器人之恋"), "声道语种"] = "其他"

In [146]:
#将完善信息后的新节目插入数据库中
new_all_program = all_program.append(new_tfv)
new_all_program.reset_index(drop=True, inplace=True)

new_all_program.to_csv("item_info_new.csv", index=False)

# get user_item

In [24]:
def change_sec(time_str):
    hmc = time_str.split(":")
    sec = int(hmc[0])*60*60 + int(hmc[1])*60 + int(hmc[2])
    return sec

def change_time(time_sec):
    sec = 0
    minu = 0
    hour = 0
    if time_sec < 60:
        sec = int(time_sec)
    elif time_sec < 3600:
        a = divmod(time_sec, 60)
        minu = int(a[0])
        sec = int(a[1])
    else:
        b = divmod(time_sec, 60*60)
        hour = int(b[0])
        if b[1] < 60:
            sec = int(b[1])
        else:
            c = divmod(b[1], 60)
            minu = int(c[0])
            sec = int(c[1])
    time = "{}:{}:{}".format(hour, minu, sec)
    return time      

In [25]:
import time
start = time.time()

user_play_1 = add_feature(user_play_1)
user_play_2 = add_feature(user_play_2)
#x新增一列【收看时长（秒）】
user_play_2["收看时长（秒）"] = [change_sec(str(i)) for i in user_play_2["收看时长"]]



In [26]:
a = user_play_2.groupby(by=["设备号", "节目名称", "形式", 
                            "地区参数","声道语种","类型"], as_index=False)

users_list = []
for i, j in a:
    users = {}
    users["user"] = i[0]
    users["program"] = i[1]
    users["form"] = i[2]
    users["location"] = i[3]
    users["language"] = i[4]
    users["tag"] = i[5]
    time_s = j.sort_values(by="收看时长（秒）", ascending=False)["收看时长（秒）"].sum()
    avg_time_s = j.sort_values(by="收看时长（秒）", ascending=False)["收看时长（秒）"].sum()/ len(j.sort_values(by="收看时长（秒）", ascending=False))
    users["avg_time_s"] = avg_time_s
    users["time_s"] = time_s
    users_list.append(users)

In [27]:
user_play_2_2 = pd.DataFrame(users_list)

In [28]:
#单片产品平均收看时长的描述性统计
ad_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="广告")].describe()["avg_time_s"]
life_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="生活服务")].describe()["avg_time_s"]
cartoon_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="动画")].describe()["avg_time_s"]
tv_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="电视剧")].describe()["avg_time_s"]
science_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="科学教育")].describe()["avg_time_s"]
music_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="音乐")].describe()["avg_time_s"]
film_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="电影")].describe()["avg_time_s"]
variety_desc2 = user_play_2_2.loc[(user_play_2_2["form"]=="综艺")].describe()["avg_time_s"]

In [29]:
for i in user_play_1['形式']:
    user_play_1.loc[user_play_1["形式"] == i, "time_s"] = user_play_2_2[user_play_2_2["form"]==i]["avg_time_s"].mean()

In [30]:
user_program_1 = user_play_1[["设备号", "节目名称", "形式", "地区参数", "声道语种", "类型", "time_s"]]
user_program_2 = user_play_2[["设备号", "节目名称", "形式", "地区参数", "声道语种", "类型", "收看时长（秒）"]]
user_program_1.rename(columns={"设备号":"user","节目名称":"program","地区参数":"location","声道语种":"language","类型":"tag","形式":"form"},inplace=True)
user_program_2.rename(columns={"设备号":"user","节目名称":"program","地区参数":"location","声道语种":"language","类型":"tag","形式":"form","收看时长（秒）":"time_s"},inplace=True)
user_program = user_program_1.append(user_program_2)
user_program.reset_index(drop=True, inplace=True)

c:\users\huashaojie\appdata\local\programs\python\python35\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [31]:
end = time.time()
print("用时：",end-start)

用时： 66.59314942359924


In [32]:
user_program.to_csv("user_item.csv", index=False)